In [1]:
import json_lines
from langdetect import detect
from langdetect import detect_langs
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()
REMOVEABLES = "123456789\/&()?:!.,;'"
ALL_TAGS = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
ACCEPTABLE_TAGS = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
MIN_ENGLISH = 0.9999
LIMIT_INPUT_ROWS = 20 # to stop run times hindering development

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [2]:
X= []; y= []; z= [];
with open ('reviews.json', 'rb') as f:
    for i, item in enumerate(json_lines.reader(f)):
        if i <= LIMIT_INPUT_ROWS:
            try:
                langs = detect_langs(item['text'])
            except:
                pass
            for i, lang in enumerate(langs):
                if str(lang)[0:2] == 'en':
                    if float(str(lang)[3:]) > MIN_ENGLISH:
                        X.append(item['text'])
                        y.append(item['voted_up'])
                        z.append(item['early_access'])

In [3]:
for i, item in enumerate(X):
    print( "X[", str(i), "]: ", str(X[i]) )
    print( "y[", str(i), "]: ", str(y[i]) )
    print( "z[", str(i), "]: ", str(z[i]) )
    print( "\n" )

X[ 0 ]:  neat little game. play if your staple fps game's server is down or sth. maps and weapon distribution system needs rework.
y[ 0 ]:  True
z[ 0 ]:  True


X[ 1 ]:  Kinda decent f2p mmorpg that has no class limitations(hooray!)(and that game where i drained too much hours...) Community: In PvE, you may encounter friendly ppl(i managed to even encounter person that i love :o), while in PvP mostly complainers(those who complain about AT&T releasing too much of 1st in battle xD) or persons that make you turn into complainer.(As always!) Main review part: Graphics: Everything looks kinda pretty, but performance is not what expected for java and openGL...(8/10) Gameplay: SK features huge choice of equipment, which adds variety to the gear and gameplay itself(like you can use guns, or bombs, or swords.. or 2 of those... or all 3 xD) But because this game features grinding in big quantities you  may get bored(or may not if you do co-op with your friend(s))(8/10) Controls: In order to rek

In [4]:
for i, text in enumerate(X):
    text_words = nltk.word_tokenize(text.lower())
    tags = nltk.pos_tag(text_words)
    print(tags)
    new_text= [];
    for j, word in enumerate(text_words):
        if tags[j][1] in ACCEPTABLE_TAGS:
            for char in word:
                if char in REMOVEABLES:
                    break
            else:
                wordnet_pos = get_wordnet_pos(tags[j][1])
                if wordnet_pos != '':
                    new_text.append(lemmatizer.lemmatize(word, wordnet_pos))
                    #print("word: ", word, ". wordnet_pos: ", wordnet_pos, "lemmad: ",lemmatizer.lemmatize(word, wordnet_pos),"\n")
                continue
    X[i] = new_text

[('neat', 'RB'), ('little', 'JJ'), ('game', 'NN'), ('.', '.'), ('play', 'VB'), ('if', 'IN'), ('your', 'PRP$'), ('staple', 'NN'), ('fps', 'JJ'), ('game', 'NN'), ("'s", 'POS'), ('server', 'NN'), ('is', 'VBZ'), ('down', 'RB'), ('or', 'CC'), ('sth', 'VB'), ('.', '.'), ('maps', 'NNS'), ('and', 'CC'), ('weapon', 'VB'), ('distribution', 'NN'), ('system', 'NN'), ('needs', 'VBZ'), ('rework', 'NN'), ('.', '.')]
[('kinda', 'VB'), ('decent', 'JJ'), ('f2p', 'NN'), ('mmorpg', 'NN'), ('that', 'WDT'), ('has', 'VBZ'), ('no', 'DT'), ('class', 'NN'), ('limitations', 'NNS'), ('(', '('), ('hooray', 'NN'), ('!', '.'), (')', ')'), ('(', '('), ('and', 'CC'), ('that', 'DT'), ('game', 'NN'), ('where', 'WRB'), ('i', 'NN'), ('drained', 'VBD'), ('too', 'RB'), ('much', 'JJ'), ('hours', 'NNS'), ('...', ':'), (')', ')'), ('community', 'NN'), (':', ':'), ('in', 'IN'), ('pve', 'NN'), (',', ','), ('you', 'PRP'), ('may', 'MD'), ('encounter', 'VB'), ('friendly', 'JJ'), ('ppl', 'NN'), ('(', '('), ('i', 'VB'), ('managed', '

In [5]:
for i, item in enumerate(X):
    print( "X[", str(i), "]: ", str(X[i]) )
    print( "y[", str(i), "]: ", str(y[i]) )
    print( "z[", str(i), "]: ", str(z[i]) )
    print( "\n" )

X[ 0 ]:  ['neat', 'little', 'game', 'play', 'staple', 'fps', 'game', 'server', 'be', 'down', 'sth', 'map', 'weapon', 'distribution', 'system', 'need', 'rework']
y[ 0 ]:  True
z[ 0 ]:  True


X[ 1 ]:  ['kinda', 'decent', 'mmorpg', 'have', 'class', 'limitation', 'hooray', 'game', 'i', 'drain', 'too', 'much', 'hour', 'community', 'pve', 'encounter', 'friendly', 'ppl', 'i', 'manage', 'even', 'encounter', 'person', 'i', 'love', 'o', 'pvp', 'mostly', 'complainer', 'complain', 't', 'release', 'too', 'much', 'battle', 'xd', 'person', 'make', 'turn', 'complainer', 'always', 'main', 'review', 'part', 'graphic', 'everything', 'look', 'kinda', 'pretty', 'performance', 'be', 'not', 'expect', 'java', 'opengl', 'gameplay', 'sk', 'feature', 'huge', 'choice', 'equipment', 'add', 'variety', 'gear', 'gameplay', 'use', 'gun', 'bomb', 'sword', 'xd', 'game', 'feature', 'grind', 'big', 'quantity', 'get', 'bore', 'not', 'do', 'co-op', 'friend', 'control', 'order', 'rekt', 'need', 'change', 'much', 'bind', 'we